# Western Cape Dam Levels
Data has been downloaded from City of Cape Town Open Data Portal:  

## GIS
City of Cape Town Corporate GIS  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::bulk-water-dams-1/explore?location=-33.865920%2C19.071866%2C11.86  
<br>
Bulk Water dams (Bulk_Water_Dams.geojson):  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::bulk-water-dams-1/explore?location=-33.865920%2C19.071866%2C11.86  
<br>
## Timeseries
Dam Levels (Dam_Levels_from_2012.csv):  
<br>
(Not Useful)  
Water Consumption (data/Water_consumption.xlsx):  
https://odp-cctegis.opendata.arcgis.com/documents/cctegis::water-consumption-1/about  
<br>
(Not Useful)  
Inland Water Quality Monthly Summary Report (Inland_WQ_Summary_Report.pdf):  
https://odp-cctegis.opendata.arcgis.com/documents/cctegis::inland-water-quality-monthly-summary-report/about  
<br>
(Not Useful)  
Rainfall Data From 2000 (Rainfall_Data_2000_to_2024.csv):  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::rainfall-data-from-2000-1/explore  

## Weather Data
meteostat python package 

## Poulations data
https://www.macrotrends.net/global-metrics/cities/22481/cape-town/population

In [ ]:
import pandas as pd
import geopandas as gpd
import json
from pandas._libs.tslibs.nattype import NaTType
import numpy as np

# Load dam polygon GeoJSON
gdf = gpd.read_file("data/Bulk_Water_Dams.geojson")

# Load dam levels CSV
df = pd.read_csv("data/Dam_Levels_from_2012.csv", encoding="ISO-8859-1")
df['DATE'] = pd.to_datetime(df['DATE'])

# Clean column names
df.columns = df.columns.str.strip().str.replace(r"\s+", "", regex=True).str.lower()

# Mapping from NAME in GeoJSON to CSV prefix (lowercase, no spaces)
dam_name_mapping = {
    "Woodhead": "woodhead",
    "Hely-Hutchinson": "hely-hutchinson",
    "Lewis Gay": "lewisgay",
    "Kleinplaats": "kleinplaats",
    "Victoria": "victoria",
    "Alexandra": "alexandra",
    "De Villiers": "devilliers",
    "Steenbras Lower": "steenbraslower",
    "Steenbras Upper": "steenbrasupper",
    "Voëlvlei": "voëlvlei",
    "Wemmershoek": "wemmershoek",
    "Theewaterskloof": "theewaterskloof",
    "Berg River": "bergriver",
    "Land-en-Zeezicht Dam": "land-enzeezicht",
    "Big 5 Total": "totalstored-big5",
    "Big 6 Total": "totalstored-big6"
}

def build_timeseries(prefix):
    # Find all columns related to this dam (that start with the prefix)
    prefix_cols = [col for col in df.columns if col.startswith(prefix)]

    def find_col(keyword):
        # Look for a column that contains the keyword (case-insensitive)
        matches = [col for col in prefix_cols if keyword in col]
        return matches[0] if matches else None

    # Find matching columns
    height_col = find_col("height")
    storage_col = find_col("storage")
    current_col = find_col("current")
    last_year_col = find_col("lastyear")

    # If we find no relevant columns, return empty
    if not any([height_col, storage_col, current_col, last_year_col]):
        return [], None

    # Build DataFrame
    cols = {'date': 'date'}
    if height_col: cols[height_col] = 'height_m'
    if storage_col: cols[storage_col] = 'storage_ml'
    if current_col: cols[current_col] = 'percent_full'
    if last_year_col: cols[last_year_col] = 'last_year_percent_full'

    col_keys = list(cols.keys())
    if 'date' not in col_keys:
        col_keys = ['date'] + col_keys
    ts = df[col_keys].copy()
    ts['date'] = pd.to_datetime(ts['date']).dt.strftime('%Y-%m-%d')  # ensure datetime
    ts.rename(columns=cols, inplace=True)

    ts = ts.where(pd.notnull(ts), None)

    current = ts.dropna(subset=['percent_full']) if 'percent_full' in ts.columns else ts
    current_percentage_full = current.sort_values("date").iloc[-1]["percent_full"] if not current.empty and 'percent_full' in current.columns else None

    return ts.to_dict(orient="records"), current_percentage_full


features = []

for _, row in gdf.iterrows():
    dam_name = row["NAME"]
    csv_prefix = dam_name_mapping.get(dam_name)
    timeseries, current_pct = build_timeseries(csv_prefix) if csv_prefix else ([], None)

    # Build GeoJSON feature
    properties = {
        k: (
            v.strftime('%Y-%m-%dT%H:%M:%SZ') if isinstance(v, pd.Timestamp)
            else None if isinstance(v, NaTType)
            else v
        )
        for k, v in row.drop("geometry").items()
    }
    properties["timeseries"] = timeseries
    properties["current_percentage_full"] = current_pct
    properties["current_date"] = max([t["date"] for t in timeseries if "date" in t], default=None)

    # Add centroid if geometry exists
    if row["geometry"]:
        centroid = row["geometry"].centroid
        properties["centroid"] = [centroid.x, centroid.y]
    else:
        properties["centroid"] = None

    feature = {
        "type": "Feature",
        "geometry": row["geometry"].__geo_interface__ if row["geometry"] else None,
        "properties": properties
    }
    features.append(feature)

# Add aggregate features (Big 5 & Big 6) without geometry
for name in ["Big 5 Total", "Big 6 Total"]:
    prefix = dam_name_mapping[name]
    timeseries, current_pct = build_timeseries(prefix)
    feature = {
        "type": "Feature",
        "geometry": None,
        "properties": {
            "NAME": name,
            "current_percentage_full": current_pct,
            "timeseries": timeseries
        }
    }
    features.append(feature)

# Build full GeoJSON
output_geojson = {
    "type": "FeatureCollection",
    "name": "SL_WTNK_BULK_DAMS_SYNC",
    "crs": {
        "type": "name",
        "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}
    },
    "features": features
}

def clean_nans(obj):
    if isinstance(obj, dict):
        return {k: clean_nans(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_nans(v) for v in obj]
    elif isinstance(obj, float) and (np.isnan(v := obj)):
        return None
    return obj

# Save
with open("output/Bulk_Water_Dams_Enriched.geojson", "w") as f:
    json.dump(clean_nans(output_geojson), f)